In [1]:
import uuid
import pandas as pd
from langchain_chroma import Chroma

from langgraph.graph import StateGraph, END

from langchain_huggingface import HuggingFaceEmbeddings, HuggingFacePipeline
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.messages import get_buffer_string
from transformers import (
    AutoTokenizer, 
    AutoModelForCausalLM, 
    AutoModelForQuestionAnswering, 
    pipeline)
from typing import Dict, Any

chats_by_ssetion_id = {}

def get_chat_history(session_id: str):
    if session_id not in chats_by_ssetion_id:
        chats_by_ssetion_id[session_id] = InMemoryChatMessageHistory()
    return chats_by_ssetion_id[session_id]

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
retriever = Chroma (
    persist_directory="./MultiAgentChroma_db",
    embedding_function=embedding_model
).as_retriever(search_kwargs={"k":3})

# KoElectra QA 모델
qa_model_id = "monologg/koelectra-base-v3-finetuned-korquad"
qa_tokenizer = AutoTokenizer.from_pretrained(qa_model_id)
qa_model = AutoModelForQuestionAnswering.from_pretrained(qa_model_id)
qa_pipeline = pipeline(
    "question-answering",
    model=qa_model,
    tokenizer=qa_tokenizer,
    device=-1
)
qa_llim = HuggingFacePipeline(pipeline=qa_pipeline)


gen_model_id = "skt/kogpt2-base-v2"
gen_tokenizer = AutoTokenizer.from_pretrained(gen_model_id)
gen_model = AutoModelForCausalLM.from_pretrained(gen_model_id)
gen_tokenizer.model_max_length = 1024
gen_pipeline = pipeline(
    "text-generation",
    model=gen_model,
    tokenizer=gen_tokenizer,
    device=-1,
    max_new_tokens=300,
    do_sample=True)

gen_llm = HuggingFacePipeline(pipeline=gen_pipeline)


def rag_node(state: Dict[str, Any]):
    mcp = state["mcp"]
    payload = mcp["payload"]
    question = payload["question"]
    sesseion_id = payload["metadata"]["session_id"]

    chat_history = get_chat_history(sesseion_id)
    history_text = "\n".join([m.content for m in chat_history.messages])
    docs = retriever.invoke(question)
    top_docs = docs[:3]
    context = "\n".join([doc.page_content for doc in top_docs])
    full_context = f"{history_text}\n{context}" if history_text else context

    result = qa_pipeline(question=question, context=full_context)
    answer = result['answer']
    chat_history.add_user_message(question)
    chat_history.add_ai_message(result['answer'])

    return {
        "mcp": {
            "source": "rag_agent",
            "destination": "textgent_agent",
            "intent":"rephrase",
            "payload": {
                "base_answer":answer,
                "metadata": {
                    "session_id": sesseion_id
                }
            }
        }
    }

def build_rag_agent():
    graph = StateGraph(dict)
    graph.set_entry_point("rag_node")
    graph.add_node("rag_node", rag_node)
    graph.set_finish_point("rag_node")
    return graph.compile()

rag_app = build_rag_agent()

def textgent_node(state: Dict[str, Any]):
    mcp = state["mcp"]
    
    answer= mcp["payload"]["base_answer"]
    sesseion_id = mcp["payload"]["metadata"]["session_id"]

    prompt = f"다음 내용을 자연스럽게 다시 표현해줘:\n{answer}"
    result = gen_llm.invoke(prompt)

    return {
        "mcp": {
            "source": "textgent_agent",
            "destination": "supervisor",
            "intent":"final_answer",
            "payload": {
                "final_answer": result,
                "metadata": {
                    "session_id": sesseion_id
                }
            }
        }
    }

def build_textgent_agent():
    graph = StateGraph(dict)
    graph.set_entry_point("textgent_node")
    graph.add_node("textgent_node", textgent_node)
    graph.set_finish_point("textgent_node")
    return graph.compile()

textgent_app = build_textgent_agent()
def handle_mcp(mcp: Dict[str, Any]):
    if mcp["destination"] == "rag_agent":
        return rag_app.invoke({"mcp": mcp})
    elif mcp["destination"] == "textgent_agent":
        return textgent_app.invoke({"mcp": mcp})
    else:
        raise ValueError(f"Unknown destination: {mcp['destination']}")
    
session_id = str(uuid.uuid4())

while True:
    question = input("\n input question:").strip()
    if not question:
        break

    mcp = {
        "source":"supervisor",
        "destination":"rag_agent",
        "intent":"get_answer",
        "payload":{
            "question": question,
            "metadata":{
                "session_id": session_id
            }
        }
    }

    result1 = handle_mcp(mcp)
    result2 = handle_mcp(result1["mcp"])

    final_answer = result2["mcp"]["payload"]["final_answer"]
    print("\n Final Answer:")
    print(final_answer)

    cont = input("\nContinue? (y/n):").strip()
    if not cont.startswith('y'):
        break
    

Device set to use cpu
Device set to use cpu



 Final Answer:
다음 내용을 자연스럽게 다시 표현해줘:
6개월간 진행된 연구에서, 실험 참가자들은 이 같은 내용을 기억하지 못했다.
실험 참가자들은 실험 동안 자신이 실제로 받은 감정 중 어떤 것이 실제로 어떤 결과를 가져오는지를 떠올렸다.
실험 후 참가자들은 실험 결과에 대한 기억이 없다는 사실을 발견했다.
그들은 실험 참가자들이 기억하지 못하는 감정들을 측정했다.
그 결과, 실험 참가자들이 경험한 감정을 기억하지 못하는 것으로 나타났다.
또한 실험 참가자들은 실험 참가자들이 실험에서 사용한 감정 중 어떤 것이 실제로 어떤 결과를 가져오는지 생각해보지도 않았다.
그 결과 실험 참가자들은 실험 참여자들이 실제 경험했던 감정 중 어떤 것이 실제로 어떠한 결과를 가져오는지 생각해보지도 못했다.
또한 실험 참가자들에게 실험 참가자들은 실험이 어떤 결과를 가져오는지 상상해보기도 하지 않았다.
즉, 실험 참가자들은 실험 참가자들이 경험한 감정들을 기억해내지 못했다.
그러나 실험 참가자들은 실제 경험하지 못하는 감정들을 상상해보지도 않았다.
그들은 실험 참가자들이 경험한 감정들을 기억하지 못했다.
즉, 실험 참가자들은 실제 경험하지 못하는 감정들을 상상해보기도 않았다.
이 같은 결과는 실험 참가자들이 실제 경험하지 못하는 감정들을 상상해보지 않았기 때문이다.
그렇다면 실험 참가자들은 실제로 경험하지 못하는 감정들을 상상해보기도 하지 않은 것일까?
이번 연구는 실험 참가자들이 실험 참가자들에게 어떤 감정을 기억하는지 알 수 있도록 해준다는 데에 의미가 있다.
즉, 실험 참가자들은 실험 참가자들이 실제 경험하지 못하는 감정들을 상상해보기도 하지 않은 채 실험 참가자들이 경험하지 못하는 감정들만을 상상해보는 것이 아니라, 실험 참가자들이 실험 참가자들이 경험하지 못하는 감정들을 상상해본다는
